In [52]:
import numpy as np
from PIL import Image
import math

In [35]:
def targetCoordToOriginalCoord(targetX,targetY):
    temp = np.array([targetX, targetY, targetX*targetY, 1])
    x, y = solveEquations()
    originalX = np.sum(np.matmul(temp, x))
    originalY = np.sum(np.matmul(temp, y))
    return originalX, originalY

In [36]:
def solveEquations():
    if not cache:
        # Ref from https://zhuanlan.zhihu.com/p/24893371
        # We have four corners, so we need four equations
        a = np.mat('0,0,0,1; {},0,0,1,; 0,{},0,1; {},{},{},1'.format(width, height, width, height, width * height))     # Reference by x = ax' + by' + cx'y' + d
        b = np.mat('{},{},{},{}'.format(coordinatesOfX[0], coordinatesOfX[1], coordinatesOfX[2], coordinatesOfX[3])).T  # Reference by y = ex' + fy' + gx'y' + h
        c = np.mat('{},{},{},{}'.format(coordinatesOfY[0], coordinatesOfY[1], coordinatesOfY[2], coordinatesOfY[3])).T  # Reference by y = ex' + fy' + gx'y' + h
        x = np.linalg.solve(a,b)
        y = np.linalg.solve(a,c)
        cache['x'] = x
        cache['y'] = y
    return cache['x'], cache['y']

In [118]:
def execBilinearInterpolation(x,y, targetX, targetY):
    # if x/y is on the point, we can skip the bilinear interpolation
    if x.is_integer() and y.is_integer():
        targetImagePixels[targetX, targetY] = pixels[x, y]
        return 
    # formula f2(x,y) = (1-a)(1-b)gs(l,k)+ a(1-b)gs(l+1,k) + (1-a)(1-b)bgs(l,k+1)+ abgs(l+1,k+1)
    
    # to prevent ceil is equal floor
    # e.g: if x is 363.0, then floor will be 363 and ceil will be 364
    ceilOfX = math.ceil(x) if math.ceil(x) != x else math.ceil(x) + 1
    ceilOfY = math.ceil(y) if math.ceil(y) != y else math.ceil(y) + 1 
    floorOfX = math.floor(x)
    floorOfY = math.floor(y)

    # calculate weight (value will be percentage)
    # Note that TL means Top left, TR means Top right, BL means Bottom left, and BR means Bottom right
    pctOfTL = (ceilOfX - x) * (ceilOfY - y)         
    pctOfTR = (x - floorOfX) * (ceilOfY - y)     
    pctOfBL = (x - floorOfX) * (y - floorOfY)  
    pctOfBR = (ceilOfX - x) * (y - floorOfY)  
    
    d2TLR, d2TLG, d2TLB = pixels[floorOfX, floorOfY]
    d2TRR, d2TRG, d2TRB = pixels[ceilOfX, floorOfY]
    d2BLR, d2BLG, d2BLB = pixels[ceilOfX, ceilOfY]
    d2BRR, d2BRG, d2BRB = pixels[floorOfX, ceilOfY]
    
    targetImagePixels[targetX, targetY] = \
    int((pctOfTL * d2TLR + pctOfTR * d2TRR + pctOfBL * d2BLR + pctOfBR * d2BRR)), \
    int((pctOfTL * d2TLG + pctOfTR * d2TRG + pctOfBL * d2BLG + pctOfBR * d2BRG)), \
    int((pctOfTL * d2TLB + pctOfTR * d2TRB + pctOfBL * d2BLB + pctOfBR * d2BRB))

In [119]:
image = Image.open('dante.jpg', "r")

# 1108 x 1478 photo frame
sourceImageWidth, sourceImageHeight = image.size
pixels = image.load()

# Set up the target photo frame
height = 300
width = 300
targetImage = Image.new("RGB", (width,height))
targetImagePixels = targetImage.load()

# coordinate of an item which wants to do the geometric transformation
# the coordinate are in order: top left, top right, bottom left, bottom right
coordinates = np.array([[320, 494], [750, 487], [278, 863], [833, 845]])
coordinatesOfX = np.array([coordinates[0][0], coordinates[1][0], coordinates[2][0], coordinates[3][0]])
coordinatesOfY = np.array([coordinates[0][1], coordinates[1][1], coordinates[2][1], coordinates[3][1]])

# cache for a solution of an equation
cache = {}

for y in range(0, height):
    for x in range(0, width):
        originalX, originalY = targetCoordToOriginalCoord(x, y)
        execBilinearInterpolation(originalX, originalY, x, y)
targetImage.save('newDante.jpg')